https://www.kaggle.com/ruchi798/movies-on-netflix-prime-video-hulu-and-disney

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rating_db = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")

In [3]:
rating_db.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [29]:
rating_db.info()

# Trocar Nan de Age por Desconhecido
# Genres - atribuir 0 para o OneHot
# Runtime - SimpleImputer
# IMDb - .dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16744 non-null  int64  
 1   ID               16744 non-null  int64  
 2   Title            16744 non-null  object 
 3   Year             16744 non-null  int64  
 4   Age              7354 non-null   object 
 5   IMDb             16173 non-null  float64
 6   Rotten Tomatoes  5158 non-null   object 
 7   Netflix          16744 non-null  int64  
 8   Hulu             16744 non-null  int64  
 9   Prime Video      16744 non-null  int64  
 10  Disney+          16744 non-null  int64  
 11  Type             16744 non-null  int64  
 12  Directors        16744 non-null  object 
 13  Genres           16469 non-null  object 
 14  Country          16744 non-null  object 
 15  Language         16744 non-null  object 
 16  Runtime          16152 non-null  float64
dtypes: float64(2

In [4]:
def splitter(x):
    try:
        return x.split(",")
    except:
        return []

In [5]:
from collections import Counter

# Não mexer:
# sorted(list(Counter(rating_db["Directors"].apply(spli).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: x[1])

In [6]:
def sorted_bom(x):
    return sorted(list(Counter(rating_db[x].apply(clacla).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

In [7]:
sorted_bom("Country")

[('United States', 10486),
 ('United Kingdom', 1713),
 ('India', 1157),
 ('Canada', 1117),
 ('France', 741),
 ('Germany', 498),
 ('Italy', 417),
 ('Spain', 329),
 ('Australia', 321),
 ('Hong Kong', 269),
 ('Japan', 266),
 ('China', 189),
 ('Mexico', 183),
 ('South Korea', 162),
 ('Belgium', 151),
 ('Ireland', 128),
 ('Sweden', 118),
 ('Netherlands', 116),
 ('Denmark', 113),
 ('Argentina', 106),
 ('Philippines', 94),
 ('Turkey', 88),
 ('South Africa', 86),
 ('Taiwan', 80),
 ('Israel', 79),
 ('Switzerland', 76),
 ('West Germany', 69),
 ('Thailand', 64),
 ('Brazil', 64),
 ('Poland', 63),
 ('Russia', 62),
 ('New Zealand', 60),
 ('Indonesia', 56),
 ('Norway', 56),
 ('Egypt', 50),
 ('Austria', 48),
 ('Czech Republic', 44),
 ('United Arab Emirates', 43),
 ('Romania', 41),
 ('Finland', 40),
 ('Hungary', 35),
 ('Colombia', 35),
 ('Greece', 32),
 ('Bulgaria', 29),
 ('Singapore', 29),
 ('Luxembourg', 27),
 ('Pakistan', 25),
 ('Iceland', 25),
 ('Nigeria', 25),
 ('Chile', 22),
 ('Portugal', 21),
 (

In [8]:
# %%time
# #Conversão dos diretores com apenas um filme para uma categoria "outros":

# directors_per_movie = rating_db.Directors.apply(clacla).apply(pd.Series)
# all_directors_contributions = sorted_bom("Directors")

# #Diretores a serem trocados
# others = []

# #Identificação dos diretores com apenas uma contribuição
# for director,contributions in all_directors_contributions:
#     if int(contributions) == 1:
#         others.append(director)

# #Listando somente diretores com mais de 2 contribuições
# official_directors = directors_per_movie.replace(others, "Outros Diretores")

Wall time: 1min 46s


In [9]:
%%time
#Conversão dos diretores com apenas um filme para uma categoria "outros":

def generaliza_valores(x, nome_coluna):
    class_per_movie = rating_db[x].apply(clacla).apply(pd.Series)
    all_directors_contributions = sorted_bom(x)

    #Diretores a serem trocados
    others = []

    #Identificação dos diretores com apenas uma contribuição
    for director,contributions in all_directors_contributions:
        if int(contributions) == 1:
            others.append(director)

    #Listando somente diretores com mais de 2 contribuições
    official_directors = directors_per_movie.replace(others, "Outros "+nome_coluna)
    return official_directors

Wall time: 0 ns


In [10]:
%%time

official_languages = generaliza_valores("Language", "Idiomas")

Wall time: 1min 51s


In [ ]:
%%time
official_directors = generaliza_valores("Directors", "Diretores")

In [11]:
%%time
official_countries = generaliza_valores("Country", "Paises")

Wall time: 7.5 s


In [12]:
# Descobrindo contagem de diretores "não-outros":
construtor1 = list(official_directors.columns)
rating_db['Directors'] = official_directors[construtor1].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)

construtor2 = list(official_languages.columns)
rating_db['Language'] = official_languages[construtor2].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)

construtor3 = list(official_languages.columns)
rating_db['Country'] = official_languages[construtor3].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)

rating_db.head(6)

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","English,Japanese,French","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",English,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",English,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",English,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,Italian,Italian,161.0
5,5,6,Spider-Man: Into the Spider-Verse,2018,7+,8.4,97%,1,0,0,0,0,"Outros Diretores,Outros Diretores,Outros Diret...","Animation,Action,Adventure,Family,Sci-Fi","English,Spanish","English,Spanish",117.0


In [13]:
# https://stackoverflow.com/questions/57469676/python-one-hot-encoding-for-comma-separated-values

In [17]:
%%time

new_df = pd.concat([rating_db.drop('Language', 1), rating_db['Language'].str.get_dummies(sep=",")], 1)
print("Rodou 1 coluna")
new_df2 = pd.concat([new_df.drop('Directors', 1), new_df['Directors'].str.get_dummies(sep=",")], 1)
print("Rodou 2 colunas")
new_df3 = pd.concat([new_df2.drop('Genres', 1), new_df2['Genres'].str.get_dummies(sep=",")], 1)
print("Rodou 3 colunas")
rating_db_tratado = pd.concat([new_df3.drop('Country', 1), new_df3['Country'].str.get_dummies(sep=",")], 1)
print("Rodou 4 colunas")

Rodou 1 coluna
Rodou 2 colunas
Rodou 3 colunas
Rodou 4 colunas
Wall time: 45.2 s


In [27]:
rating_db_tratado.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,...,Turkmen,Ukrainian,Urdu,Vietnamese,Welsh,Wolof,Xhosa,Yiddish,Yoruba,Zulu
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# https://stackoverflow.com/questions/37646473/how-could-i-do-one-hot-encoding-with-multiple-values-in-one-cell
# pd.DataFrame({"diretor":pd.Series(['2','1','3']).astype('category',categories = [sorted_bom("Directors")])})

In [ ]:
# directors = sorted(list(Counter(official_directors.unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

In [ ]:
rating_db_nans = rating_db_tratado.copy()

* Tirar as variáveis mais importantes: como fica o desempenho do regressor?
* Análise exploratória - visualização
* Criar uma lista de perguntas para responder - ex. qual diretor tem mais impacto positivo no rating? Qual a relação entre rating e ano de lançamento?
* Qualidade do código - melhorar nome das variáveis